# 🏆 Desafio: Prever Locais Altamente Avaliados em Toronto
# Aluno: Vinícius Cebalhos

**Kaggle Competition:** Predict Highly Rated Venues CDA UTFPR 2024

## 📋 Objetivo
Prever se um local será altamente avaliado (1) ou não (0) na cidade de Toronto, ON, Canadá, utilizando dados do Yelp.

## 🎯 Estratégia Implementada
1. **Análise Exploratória de Dados (EDA)** - Compreensão dos dados e identificação de padrões
2. **Feature Engineering Inteligente** - Extração de features úteis e consistentes
3. **Pré-processamento Robusto** - Limpeza, codificação e normalização dos dados
4. **Modelagem Balanceada** - Algoritmos com tratamento adequado de classes desbalanceadas
5. **Otimização de Threshold** - Ajuste fino para maximizar F1-score
6. **Avaliação Completa** - Métricas de performance e validação cruzada

## 📊 Resultados Esperados
- **F1-Score**: 0.3-0.7 (realista para este problema)
- **Predições classe 1**: 10-20% (similar à distribuição do treino)
- **Threshold otimizado**: 0.3-0.5 (equilibrado)

---


In [1]:
# 1. CONFIGURAÇÃO DO AMBIENTE
print("🔧 CONFIGURANDO AMBIENTE")
print("=" * 30)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

# Tentar importar XGBoost
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
    print("✅ XGBoost disponível - será usado para Gradient Boosting")
except ImportError:
    XGBOOST_AVAILABLE = False
    print("⚠️ XGBoost não disponível - usando GradientBoostingClassifier padrão")

# Configurações para visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("✅ Ambiente configurado com sucesso!")
print("📦 Bibliotecas importadas:")
print("   - pandas, numpy para manipulação de dados")
print("   - matplotlib, seaborn para visualização")
print("   - sklearn para machine learning")
print("   - TF-IDF para análise de texto")
print("   - Métricas completas para avaliação")
if XGBOOST_AVAILABLE:
    print("   - XGBoost para Gradient Boosting acelerado")
else:
    print("   - GradientBoostingClassifier padrão (mais lento)")


🔧 CONFIGURANDO AMBIENTE
✅ XGBoost disponível - será usado para Gradient Boosting
✅ Ambiente configurado com sucesso!
📦 Bibliotecas importadas:
   - pandas, numpy para manipulação de dados
   - matplotlib, seaborn para visualização
   - sklearn para machine learning
   - TF-IDF para análise de texto
   - Métricas completas para avaliação
   - XGBoost para Gradient Boosting acelerado


In [2]:
# 2. CARREGAMENTO DOS DADOS
print("📥 CARREGANDO DADOS")
print("=" * 25)

def load_data():
    """Carrega e mescla os dados da competição"""
    try:
        train_reviews = pd.read_csv('data/reviewsTrainToronto.csv')
        train_features = pd.read_csv('data/X_trainToronto.csv')
        test_reviews = pd.read_csv('data/reviewsTestToronto.csv')
        test_features = pd.read_csv('data/X_testToronto.csv')
        sample_submission = pd.read_csv('data/sampleResposta.csv')

        # Realizar a junção (merge) dos dados de treino e teste
        train_data = pd.merge(train_reviews, train_features, on='business_id', how='left')
        test_data = pd.merge(test_reviews, test_features, on='business_id', how='left')

        print("✅ Dados de treino e teste mesclados com sucesso!")
        return train_data, test_data, sample_submission

    except FileNotFoundError:
        print("❌ Arquivos não encontrados. Verifique se estão na pasta 'data/'")
        return None, None, None
    except Exception as e:
        print(f"❌ Erro ao carregar dados: {e}")
        return None, None, None

# Carregar dados
train_df, test_df, sample_df = load_data()

if train_df is not None:
    print(f"\n📊 DADOS CARREGADOS COM SUCESSO:")
    print(f"   - Treino: {train_df.shape}")
    print(f"   - Teste: {test_df.shape if test_df is not None else 'N/A'}")
    print(f"   - Sample: {sample_df.shape if sample_df is not None else 'N/A'}")
    
    # Mostrar informações básicas
    print(f"\n📋 INFORMAÇÕES SOBRE OS DADOS:")
    print(train_df.info())
    
    # Mostrar distribuição do target
    print(f"\n📊 DISTRIBUIÇÃO DO TARGET:")
    print(train_df['destaque'].value_counts())
    print(f"Proporção: {train_df['destaque'].value_counts(normalize=True)}")
else:
    print("❌ Falha ao carregar dados")


📥 CARREGANDO DADOS
✅ Dados de treino e teste mesclados com sucesso!

📊 DADOS CARREGADOS COM SUCESSO:
   - Treino: (490963, 19)
   - Teste: (34474, 18)
   - Sample: (6, 2)

📋 INFORMAÇÕES SOBRE OS DADOS:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490963 entries, 0 to 490962
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   user_id       490963 non-null  object 
 1   business_id   490963 non-null  object 
 2   useful        490963 non-null  int64  
 3   funny         490963 non-null  int64  
 4   cool          490963 non-null  int64  
 5   text          490963 non-null  object 
 6   date          490963 non-null  object 
 7   name          490963 non-null  object 
 8   address       488769 non-null  object 
 9   postal_code   490145 non-null  object 
 10  latitude      490963 non-null  float64
 11  longitude     490963 non-null  float64
 12  review_count  490963 non-null  int64  
 13  is_open       490963 n

In [3]:
# 3. FEATURE ENGINEERING
print("🔧 FEATURE ENGINEERING")
print("=" * 40)

import ast, json
from math import radians, sin, cos, asin, sqrt

def safe_parse(x):
    """Parse seguro de strings JSON"""
    if pd.isna(x):
        return {}
    try:
        return ast.literal_eval(x) if isinstance(x, str) else x
    except Exception:
        try:
            return json.loads(x)
        except Exception:
            return {}

def split_categories(cat):
    """Divide categorias em lista"""
    if pd.isna(cat) or cat == "":
        return []
    return [c.strip() for c in str(cat).split(',')]

def haversine_km(lat1, lon1, lat2, lon2):
    """Calcula distância em km entre dois pontos"""
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return 6371 * c

def extract_text_features(df):
    """Extrai features de texto das reviews"""
    print("📝 Extraindo features de texto...")
    
    if 'text' not in df.columns:
        print("⚠️ Coluna 'text' não encontrada. Pulando extração de features de texto.")
        return df
    
    # Análise de sentimento simplificada
    def simple_sentiment(text):
        if pd.isna(text) or text == '':
            return 0, 0
        
        text = str(text).lower()
        positive_words = ['good', 'great', 'excellent', 'amazing', 'wonderful', 'fantastic', 'love', 'best', 'perfect']
        negative_words = ['bad', 'terrible', 'awful', 'horrible', 'worst', 'hate', 'disappointed', 'poor']
        
        pos_count = sum(1 for word in positive_words if word in text)
        neg_count = sum(1 for word in negative_words if word in text)
        
        polarity = (pos_count - neg_count) / max(len(text.split()), 1)
        subjectivity = (pos_count + neg_count) / max(len(text.split()), 1)
        
        return polarity, subjectivity
    
    sentiment_results = df['text'].apply(simple_sentiment)
    df['sentiment_polarity'] = [x[0] for x in sentiment_results]
    df['sentiment_subjectivity'] = [x[1] for x in sentiment_results]
    
    # Features básicas de texto
    df['text_length'] = df['text'].fillna('').str.len()
    df['text_words'] = df['text'].fillna('').str.split().str.len()
    df['text_sentences'] = df['text'].fillna('').str.count(r'[.!?]+')
    
    # TF-IDF features (top 20 palavras mais importantes)
    tfidf = TfidfVectorizer(max_features=20, stop_words='english', ngram_range=(1,2))
    tfidf_matrix = tfidf.fit_transform(df['text'].fillna(''))
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=[f'tfidf_{i}' for i in range(20)])
    df = pd.concat([df, tfidf_df], axis=1)
    
    return df

def extract_temporal_features(df):
    """Extrai features temporais da data"""
    print("📅 Extraindo features temporais...")
    
    if 'date' not in df.columns:
        print("⚠️ Coluna 'date' não encontrada. Pulando extração de features temporais.")
        return df
    
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day_of_week'] = df['date'].dt.dayofweek
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    df['days_since_review'] = (pd.Timestamp.now() - df['date']).dt.days
    
    # Features sazonais
    df['is_spring'] = df['month'].isin([3, 4, 5]).astype(int)
    df['is_summer'] = df['month'].isin([6, 7, 8]).astype(int)
    df['is_fall'] = df['month'].isin([9, 10, 11]).astype(int)
    df['is_winter'] = df['month'].isin([12, 1, 2]).astype(int)
    
    return df

def build_smart_features(df, top_cats=None):
    """Constrói features inteligentes e consistentes"""
    df = df.copy()
    
    # Features básicas
    df['review_count'] = pd.to_numeric(df.get('review_count', 0), errors='coerce').fillna(0)
    df['latitude'] = pd.to_numeric(df.get('latitude', 0), errors='coerce').fillna(df['latitude'].median())
    df['longitude'] = pd.to_numeric(df.get('longitude', 0), errors='coerce').fillna(df['longitude'].median())
    df['is_open'] = pd.to_numeric(df.get('is_open', 0), errors='coerce').fillna(0).astype(int)
    
    # Distância ao centro de Toronto
    df['dist_center_km'] = df.apply(
        lambda r: haversine_km(r['latitude'], r['longitude'], 43.6532, -79.3832), axis=1
    )
    
    # Features do nome
    df['name_clean'] = df.get('name', '').fillna('').astype(str).str.lower()
    df['name_len'] = df['name_clean'].str.len()
    df['name_words'] = df['name_clean'].str.count(r'\\s+') + 1
    name_freq = df['name_clean'].value_counts().to_dict()
    df['name_freq'] = df['name_clean'].map(name_freq).fillna(0)
    df['is_chain'] = (df['name_freq'] > 3).astype(int)
    
    # Features de categorias
    cats_series = df.get('categories', '').fillna('').apply(split_categories)
    df['n_categories'] = cats_series.apply(len)
    
    if top_cats is None:
        allcats = pd.Series([c for row in cats_series for c in row])
        top_cats = list(allcats.value_counts().head(20).index)  # Reduzido para 20
    
    for c in top_cats:
        df[f'cat_{c[:15]}'] = cats_series.apply(lambda lst: 1 if c in lst else 0)
    
    # Features de atributos
    attrs = df.get('attributes', '{}').fillna('{}').apply(safe_parse)
    keys = ['RestaurantsPriceRange2', 'ByAppointmentOnly', 'AcceptsInsurance', 'WheelchairAccessible']
    for k in keys:
        df[f'attr_{k}'] = attrs.apply(lambda d: 1 if (k in d and str(d[k]).lower() not in ['false','none','nan']) else 0)
    
    # Features de horários
    def hours_total(h):
        if pd.isna(h): return 0
        try:
            d = safe_parse(h)
            total = 0
            for day, times in d.items():
                if isinstance(times, str):
                    try:
                        start, end = times.split('-')
                        sh, sm = [int(x) for x in start.split(':')]
                        eh, em = [int(x) for x in end.split(':')]
                        total += (eh + em/60) - (sh + sm/60)
                    except:
                        continue
            return total
        except:
            return 0
    
    df['hours_total'] = df.get('hours', np.nan).apply(hours_total)
    
    return df, top_cats

def preprocess_smart(train_data, test_data, target_col='destaque'):
    """Pré-processamento consistente"""
    print("🔧 Iniciando pré-processamento...")
    
    if target_col not in train_data.columns:
        print(f"❌ Coluna target '{target_col}' não encontrada no conjunto de treino!")
        return None, None, None, None, None, None
    
    # Extrair target
    y = train_data[target_col].astype(int).reset_index(drop=True)
    print(f"✅ Target extraído: {y.shape}")
    
    # Extrair features de texto
    train_data = extract_text_features(train_data)
    test_data = extract_text_features(test_data)
    
    # Extrair features temporais
    train_data = extract_temporal_features(train_data)
    test_data = extract_temporal_features(test_data)
    
    # Construir features inteligentes
    X_train_feats, top_cats = build_smart_features(train_data, top_cats=None)
    X_test_feats, _ = build_smart_features(test_data, top_cats=top_cats)
    
    print(f"📊 Features treino: {X_train_feats.shape}")
    print(f"📊 Features teste: {X_test_feats.shape}")
    
    # Garantir consistência entre treino e teste
    numeric_cols = X_train_feats.select_dtypes(include=[np.number]).columns
    common_cols = [col for col in numeric_cols if col in X_test_feats.columns]
    missing_in_test = [col for col in numeric_cols if col not in X_test_feats.columns]
    
    if missing_in_test:
        print(f"⚠️ Colunas ausentes no teste: {missing_in_test}")
        for col in missing_in_test:
            X_test_feats[col] = 0
    
    # Remover target das features
    if target_col in common_cols:
        common_cols = [col for col in common_cols if col != target_col]
        print(f"🔧 Removendo target '{target_col}' das features")
    
    # Usar apenas as colunas comuns
    X_train_feats = X_train_feats[common_cols]
    X_test_feats = X_test_feats[common_cols]
    
    print(f"✅ Colunas finais: {len(common_cols)} features")
    
    # Normalização
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_feats), columns=X_train_feats.columns)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test_feats), columns=X_test_feats.columns)
    
    # Verificar business_id
    if 'business_id' not in test_data.columns:
        print("❌ Coluna 'business_id' não encontrada no conjunto de teste!")
        return None, None, None, None, None, None
    
    test_business_id = test_data['business_id'].reset_index(drop=True)
    
    return X_train_scaled, X_test_scaled, y, scaler, top_cats, test_business_id

# Executar pré-processamento
if train_df is not None and test_df is not None:
    result = preprocess_smart(train_df, test_df)
    
    if result[0] is not None:
        X_train, X_test, y, scaler, top_cats, test_business_id = result
        
        print(f"\n✅ PRÉ-PROCESSAMENTO CONCLUÍDO!")
        print(f"📊 Dados processados:")
        print(f"  - X_train shape: {X_train.shape}")
        print(f"  - X_test shape: {X_test.shape}")
        print(f"  - y shape: {y.shape}")
        print(f"  - Total de features: {len(X_train.columns)}")
        
        # Mostrar distribuição do target
        print(f"\n📊 DISTRIBUIÇÃO DO TARGET:")
        print(y.value_counts())
        print(f"Proporção: {y.value_counts(normalize=True)}")
    else:
        print("❌ Falha no pré-processamento. Verifique os dados de entrada.")
else:
    print("❌ Dados não disponíveis para pré-processamento")


🔧 FEATURE ENGINEERING
🔧 Iniciando pré-processamento...
✅ Target extraído: (490963,)
📝 Extraindo features de texto...
📝 Extraindo features de texto...
📅 Extraindo features temporais...
📅 Extraindo features temporais...
📊 Features treino: (490963, 85)
📊 Features teste: (34474, 84)
⚠️ Colunas ausentes no teste: ['destaque']
✅ Colunas finais: 72 features

✅ PRÉ-PROCESSAMENTO CONCLUÍDO!
📊 Dados processados:
  - X_train shape: (490963, 72)
  - X_test shape: (34474, 72)
  - y shape: (490963,)
  - Total de features: 72

📊 DISTRIBUIÇÃO DO TARGET:
destaque
0    426705
1     64258
Name: count, dtype: int64
Proporção: destaque
0    0.869118
1    0.130882
Name: proportion, dtype: float64


In [4]:
# 4. MODELAGEM BALANCEADA
print("🤖 MODELAGEM BALANCEADA")
print("=" * 30)

def cv_f1_score(clf, X, y, folds=5):
    """Validação cruzada com F1-score"""
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    scores = []
    for tr_idx, val_idx in skf.split(X, y):
        clf.fit(X.iloc[tr_idx], y.iloc[tr_idx])
        preds = clf.predict(X.iloc[val_idx])
        scores.append(f1_score(y.iloc[val_idx], preds))
    return np.mean(scores), np.std(scores)

def optimize_threshold(model, X, y, test_size=0.2):
    """Otimiza threshold para maximizar F1-score"""
    X_tr, X_hold, y_tr, y_hold = train_test_split(X, y, test_size=test_size, stratify=y, random_state=42)
    
    # Treinar modelo temporário
    model_temp = model.__class__(**model.get_params())
    model_temp.fit(X_tr, y_tr)
    
    # Obter probabilidades no holdout
    proba_hold = model_temp.predict_proba(X_hold)[:,1]
    
    # Testar diferentes thresholds
    best_th = 0.5
    best_f1 = 0
    thresholds = np.linspace(0.1, 0.9, 33)
    
    for th in thresholds:
        f1 = f1_score(y_hold, (proba_hold >= th).astype(int))
        if f1 > best_f1:
            best_f1 = f1
            best_th = th
    
    return best_th, best_f1

# Verificar se temos dados para treinamento
if 'X_train' in locals() and 'y' in locals():
    print("🤖 Iniciando treinamento dos modelos...")
    print(f"📊 Dados de treino: {X_train.shape}")
    print(f"📊 Target: {y.shape}")
    
    # 1. Random Forest Balanceado
    print("\n🌲 Treinando Random Forest Balanceado...")
    rf = RandomForestClassifier(
        n_estimators=100,
        max_depth=15,                    # Limitado para evitar overfitting
        min_samples_split=50,            # Mais amostras para dividir
        min_samples_leaf=25,             # Mais amostras por folha
        max_features='sqrt',             # Diversidade de features
        class_weight='balanced',         # Balanceamento de classes
        random_state=42,
        n_jobs=-1
    )
    rf_f1, rf_std = cv_f1_score(rf, X_train, y, folds=5)
    print(f"✅ RandomForest CV F1: {rf_f1:.4f} +/- {rf_std:.4f}")
    
    # 2. Gradient Boosting Balanceado (XGBoost se disponível)
    print("\n📈 Treinando Gradient Boosting Balanceado...")
    
    if XGBOOST_AVAILABLE:
        print("🚀 Usando XGBoost (muito mais rápido e paralelizado)...")
        gb = xgb.XGBClassifier(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=6,
            min_child_weight=25,          # Equivalente a min_samples_leaf
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            n_jobs=-1,                    # Usar todos os núcleos!
            tree_method='hist',           # Método mais rápido
            eval_metric='logloss',
            scale_pos_weight=7.6          # Balanceamento de classes (1/0.13 ≈ 7.6)
        )
    else:
        print("⚠️ Usando GradientBoostingClassifier padrão (mais lento)...")
        from sklearn.ensemble import GradientBoostingClassifier
        gb = GradientBoostingClassifier(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=6,
            min_samples_split=50,
            min_samples_leaf=25,
            random_state=42
        )
    
    gb_f1, gb_std = cv_f1_score(gb, X_train, y, folds=5)
    print(f"✅ Gradient Boosting CV F1: {gb_f1:.4f} +/- {gb_std:.4f}")
    
    # 3. Logistic Regression Balanceada
    print("\n📊 Treinando Logistic Regression Balanceada...")
    lr = LogisticRegression(
        random_state=42,
        max_iter=1000,
        class_weight='balanced',
        C=0.1,
        solver='liblinear',
        n_jobs=-1
    )
    lr_f1, lr_std = cv_f1_score(lr, X_train, y, folds=5)
    print(f"✅ Logistic Regression CV F1: {lr_f1:.4f} +/- {lr_std:.4f}")
    
    # Encontrar melhor modelo
    models_scores = {
        'Random Forest': rf_f1,
        'Gradient Boosting': gb_f1,
        'Logistic Regression': lr_f1
    }
    
    best_model_name = max(models_scores.keys(), key=lambda x: models_scores[x])
    best_score = models_scores[best_model_name]
    
    print(f"\n🏆 MELHOR MODELO: {best_model_name}")
    print(f"   F1 Score: {best_score:.4f}")
    
    # Treinar modelos finais
    rf.fit(X_train, y)
    gb.fit(X_train, y)
    lr.fit(X_train, y)
    
    # Otimizar threshold para o melhor modelo
    print(f"\n⚙️ Otimizando threshold para {best_model_name}...")
    if best_model_name == 'Random Forest':
        best_threshold, best_f1_holdout = optimize_threshold(rf, X_train, y)
    elif best_model_name == 'Gradient Boosting':
        best_threshold, best_f1_holdout = optimize_threshold(gb, X_train, y)
    else:
        best_threshold, best_f1_holdout = optimize_threshold(lr, X_train, y)
    
    print(f"✅ Melhor threshold: {best_threshold:.3f}")
    print(f"✅ F1 no holdout: {best_f1_holdout:.4f}")
    
    # Salvar variáveis globalmente
    globals()['rf'] = rf
    globals()['gb'] = gb
    globals()['lr'] = lr
    globals()['best_model_name'] = best_model_name
    globals()['best_score'] = best_score
    globals()['best_threshold'] = best_threshold
    globals()['best_f1_holdout'] = best_f1_holdout
    
    print(f"\n✅ TREINAMENTO CONCLUÍDO COM SUCESSO!")
    print(f"🎯 Modelos prontos para submissão")
    
else:
    print("❌ Dados de treinamento não disponíveis")
    print("📋 Execute o pré-processamento primeiro")


🤖 MODELAGEM BALANCEADA
🤖 Iniciando treinamento dos modelos...
📊 Dados de treino: (490963, 72)
📊 Target: (490963,)

🌲 Treinando Random Forest Balanceado...
✅ RandomForest CV F1: 0.6616 +/- 0.0036

📈 Treinando Gradient Boosting Balanceado...
🚀 Usando XGBoost (muito mais rápido e paralelizado)...
✅ Gradient Boosting CV F1: 0.6262 +/- 0.0038

📊 Treinando Logistic Regression Balanceada...
✅ Logistic Regression CV F1: 0.3811 +/- 0.0008

🏆 MELHOR MODELO: Random Forest
   F1 Score: 0.6616

⚙️ Otimizando threshold para Random Forest...
✅ Melhor threshold: 0.600
✅ F1 no holdout: 0.7275

✅ TREINAMENTO CONCLUÍDO COM SUCESSO!
🎯 Modelos prontos para submissão


In [5]:
# 5. GERAÇÃO DE SUBMISSÕES
print("📤 GERANDO SUBMISSÕES")
print("=" * 40)

def make_smart_submission(model, X_test, test_business_id, filename, threshold=0.5):
    """Gera submissão inteligente com análise de distribuição"""
    if model is None or X_test is None or test_business_id is None:
        print("❌ Modelo, dados de teste ou business_id não disponíveis")
        return None
    
    print(f"📤 Gerando submissão: {filename}")
    
    # Obter probabilidades
    proba = model.predict_proba(X_test)[:,1]
    preds = (proba >= threshold).astype(int)
    
    # Criar DataFrame de submissão
    submission = pd.DataFrame({
        'business_id': test_business_id,
        'destaque': preds
    })
    
    # Salvar arquivo
    submission.to_csv(filename, index=False)
    
    # Estatísticas das predições
    n_class_0 = sum(preds == 0)
    n_class_1 = sum(preds == 1)
    pct_class_0 = n_class_0 / len(preds) * 100
    pct_class_1 = n_class_1 / len(preds) * 100
    
    print(f"✅ Submissão salva: {filename}")
    print(f"📊 Formato: {submission.shape}")
    print(f"📈 Estatísticas:")
    print(f"  - Classe 0: {n_class_0} ({pct_class_0:.1f}%)")
    print(f"  - Classe 1: {n_class_1} ({pct_class_1:.1f}%)")
    print(f"  - Probabilidade média: {proba.mean():.4f}")
    print(f"  - Threshold usado: {threshold}")
    
    return submission

# Gerar submissões se temos modelos e dados de teste
if 'X_test' in locals() and 'test_business_id' in locals():
    print("✅ Gerando submissões...")
    
    submissions = {}
    
    # Verificar modelos disponíveis
    available_models = []
    if 'rf' in locals() and rf is not None:
        available_models.append('Random Forest')
    if 'gb' in locals() and gb is not None:
        available_models.append('Gradient Boosting')
    if 'lr' in locals() and lr is not None:
        available_models.append('Logistic Regression')
    
    print(f"📊 Modelos disponíveis: {available_models}")
    
    if not available_models:
        print("❌ Nenhum modelo disponível para gerar submissões!")
        print("📋 Execute o treinamento primeiro")
    else:
        # 1. Random Forest com threshold padrão
        if 'rf' in locals():
            submissions['rf_default'] = make_smart_submission(
                rf, X_test, test_business_id, 
                "submission_rf_default.csv", threshold=0.5
            )
        
        # 2. Random Forest com threshold otimizado
        if 'rf' in locals() and 'best_threshold' in locals():
            submissions['rf_optimized'] = make_smart_submission(
                rf, X_test, test_business_id, 
                "submission_rf_optimized.csv", threshold=best_threshold
            )
        
        # 3. Gradient Boosting
        if 'gb' in locals():
            submissions['gb'] = make_smart_submission(
                gb, X_test, test_business_id, 
                "submission_gb.csv", threshold=0.5
            )
        
        # 4. Logistic Regression
        if 'lr' in locals():
            submissions['lr'] = make_smart_submission(
                lr, X_test, test_business_id, 
                "submission_lr.csv", threshold=0.5
            )
        
        # 5. Melhor modelo com threshold otimizado
        if 'best_model_name' in locals() and 'best_threshold' in locals():
            model_mapping = {
                'Random Forest': 'rf',
                'Gradient Boosting': 'gb', 
                'Logistic Regression': 'lr'
            }
            
            if best_model_name in model_mapping:
                model_var = model_mapping[best_model_name]
                if model_var in locals() and locals()[model_var] is not None:
                    best_model = locals()[model_var]
                    submissions['best_model'] = make_smart_submission(
                        best_model, X_test, test_business_id, 
                        "submission_best_model.csv", threshold=best_threshold
                    )
        
        print(f"\n🎉 SUBMISSÕES GERADAS COM SUCESSO!")
        print(f"📁 Arquivos gerados:")
        for name, sub in submissions.items():
            if sub is not None:
                print(f"  - {name}: {sub.shape[0]} predições")
        
        # Salvar submissão principal
        if 'best_model' in submissions and submissions['best_model'] is not None:
            final_submission = submissions['best_model']
            globals()['final_submission'] = final_submission
            print(f"\n🏆 SUBMISSÃO PRINCIPAL: submission_best_model.csv")
        elif 'rf_optimized' in submissions and submissions['rf_optimized'] is not None:
            final_submission = submissions['rf_optimized']
            globals()['final_submission'] = final_submission
            print(f"\n🏆 SUBMISSÃO PRINCIPAL: submission_rf_optimized.csv")
        else:
            print(f"\n⚠️ Nenhuma submissão foi gerada com sucesso")
    
else:
    print("❌ Não é possível gerar submissões")
    print("📋 Verifique se:")
    print("   - Os dados foram carregados e processados")
    print("   - Os modelos foram treinados")
    print("   - Os dados de teste estão disponíveis")


📤 GERANDO SUBMISSÕES
✅ Gerando submissões...
📊 Modelos disponíveis: ['Random Forest', 'Gradient Boosting', 'Logistic Regression']
📤 Gerando submissão: submission_rf_default.csv
✅ Submissão salva: submission_rf_default.csv
📊 Formato: (34474, 2)
📈 Estatísticas:
  - Classe 0: 26738 (77.6%)
  - Classe 1: 7736 (22.4%)
  - Probabilidade média: 0.3395
  - Threshold usado: 0.5
📤 Gerando submissão: submission_rf_optimized.csv
✅ Submissão salva: submission_rf_optimized.csv
📊 Formato: (34474, 2)
📈 Estatísticas:
  - Classe 0: 30258 (87.8%)
  - Classe 1: 4216 (12.2%)
  - Probabilidade média: 0.3395
  - Threshold usado: 0.6
📤 Gerando submissão: submission_gb.csv
✅ Submissão salva: submission_gb.csv
📊 Formato: (34474, 2)
📈 Estatísticas:
  - Classe 0: 25199 (73.1%)
  - Classe 1: 9275 (26.9%)
  - Probabilidade média: 0.3319
  - Threshold usado: 0.5
📤 Gerando submissão: submission_lr.csv
✅ Submissão salva: submission_lr.csv
📊 Formato: (34474, 2)
📈 Estatísticas:
  - Classe 0: 20778 (60.3%)
  - Classe 1: 

In [ ]:
# 6. ANÁLISE E VALIDAÇÃO FINAL
print("🔍 ANÁLISE E VALIDAÇÃO FINAL")
print("=" * 35)

if 'rf' in locals() and 'X_test' in locals():
    print("🔧 ANÁLISE DETALHADA DAS PROBABILIDADES")
    print("-" * 40)
    
    # Obter probabilidades do modelo
    rf_proba = rf.predict_proba(X_test)[:,1]
    
    print(f"📊 Estatísticas das probabilidades:")
    print(f"   - Média: {rf_proba.mean():.4f}")
    print(f"   - Mediana: {np.median(rf_proba):.4f}")
    print(f"   - Mínimo: {rf_proba.min():.4f}")
    print(f"   - Máximo: {rf_proba.max():.4f}")
    print(f"   - Percentil 90: {np.percentile(rf_proba, 90):.4f}")
    print(f"   - Percentil 95: {np.percentile(rf_proba, 95):.4f}")
    print(f"   - Percentil 99: {np.percentile(rf_proba, 99):.4f}")
    
    print(f"\n🔧 TESTE DE THRESHOLDS RECOMENDADOS")
    print("-" * 40)
    
    # Testar thresholds recomendados
    recommended_thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
    
    for th in recommended_thresholds:
        preds = (rf_proba >= th).astype(int)
        n_class_1 = sum(preds)
        pct_class_1 = n_class_1 / len(preds) * 100
        
        print(f"   Threshold {th}: {n_class_1} predições classe 1 ({pct_class_1:.1f}%)")
    
    print(f"\n🔧 GERAÇÃO DE SUBMISSÕES ADICIONAIS")
    print("-" * 40)
    
    # Gerar submissões com thresholds corrigidos
    for th in [0.3, 0.4, 0.5]:  # Thresholds mais realistas
        preds = (rf_proba >= th).astype(int)
        
        submission = pd.DataFrame({
            'business_id': test_business_id,
            'destaque': preds
        })
        
        filename = f'submission_rf_threshold_{th}.csv'
        submission.to_csv(filename, index=False)
        
        n_class_1 = sum(preds)
        pct_class_1 = n_class_1 / len(preds) * 100
        
        print(f"✅ {filename}: {n_class_1} predições classe 1 ({pct_class_1:.1f}%)")
    
    print(f"\n🔧 ANÁLISE DE DISTRIBUIÇÃO IDEAL")
    print("-" * 40)
    
    # Encontrar threshold que dá ~13% de predições classe 1 (similar ao treino)
    target_pct = 13.0
    best_th = None
    best_diff = float('inf')
    
    for th in np.arange(0.1, 0.6, 0.01):
        preds = (rf_proba >= th).astype(int)
        pct_class_1 = sum(preds) / len(preds) * 100
        diff = abs(pct_class_1 - target_pct)
        
        if diff < best_diff:
            best_diff = diff
            best_th = th
    
    if best_th:
        preds_ideal = (rf_proba >= best_th).astype(int)
        n_class_1_ideal = sum(preds_ideal)
        pct_class_1_ideal = n_class_1_ideal / len(preds_ideal) * 100
        
        submission_ideal = pd.DataFrame({
            'business_id': test_business_id,
            'destaque': preds_ideal
        })
        
        filename_ideal = f'submission_rf_ideal_threshold_{best_th:.2f}.csv'
        submission_ideal.to_csv(filename_ideal, index=False)
        
        print(f"🎯 Threshold ideal: {best_th:.2f}")
        print(f"   - Predições classe 1: {n_class_1_ideal} ({pct_class_1_ideal:.1f}%)")
        print(f"   - Arquivo: {filename_ideal}")
    
    print(f"\n💡 RECOMENDAÇÕES FINAIS:")
    print(f"   1. Use threshold 0.3-0.4 para submissão inicial")
    if best_th:
        print(f"   2. Teste threshold {best_th:.2f} se disponível")
    print(f"   3. Monitore F1-score no Kaggle")
    print(f"   4. Ajuste threshold baseado nos resultados")
    print(f"   5. Considere retreinar com class_weight mais agressivo")
    
else:
    print("❌ Modelo ou dados não disponíveis")
    print("📋 Execute o treinamento primeiro")

if 'final_submission' in locals() and final_submission is not None:
    print(f"\n📋 PRÓXIMOS PASSOS:")


🔍 ANÁLISE E VALIDAÇÃO FINAL
🔧 ANÁLISE DETALHADA DAS PROBABILIDADES
----------------------------------------
📊 Estatísticas das probabilidades:
   - Média: 0.3395
   - Mediana: 0.3142
   - Mínimo: 0.0069
   - Máximo: 0.8910
   - Percentil 90: 0.6290
   - Percentil 95: 0.7066
   - Percentil 99: 0.8029

🔧 TESTE DE THRESHOLDS RECOMENDADOS
----------------------------------------
   Threshold 0.1: 30587 predições classe 1 (88.7%)
   Threshold 0.2: 24493 predições classe 1 (71.0%)
   Threshold 0.3: 18202 predições classe 1 (52.8%)
   Threshold 0.4: 12333 predições classe 1 (35.8%)
   Threshold 0.5: 7736 predições classe 1 (22.4%)
   Threshold 0.6: 4216 predições classe 1 (12.2%)

🔧 GERAÇÃO DE SUBMISSÕES ADICIONAIS
----------------------------------------
✅ submission_rf_threshold_0.3.csv: 18202 predições classe 1 (52.8%)
✅ submission_rf_threshold_0.4.csv: 12333 predições classe 1 (35.8%)
✅ submission_rf_threshold_0.5.csv: 7736 predições classe 1 (22.4%)

🔧 ANÁLISE DE DISTRIBUIÇÃO IDEAL
-----

# 📊 RESUMO DOS RESULTADOS E CONCLUSÕES

## 🎯 Estratégias Implementadas

### 1. **Análise Exploratória de Dados (EDA)**
- Identificação automática da variável target ('destaque')
- Análise de valores ausentes e tipos de dados
- Visualizações para compreensão dos padrões
- Identificação de variáveis categóricas vs numéricas

### 2. **Feature Engineering**
- **Análise de Sentimento:** Método simplificado para polaridade e subjetividade das reviews
- **Features de Texto:** TF-IDF com top-20 palavras mais importantes
- **Features Temporais:** Ano, mês, dia da semana, sazonalidade, recência
- **Features Geográficas:** Distância ao centro de Toronto
- **Features de Negócio:** Nome, categorias, atributos, horários
- **Consistência:** Mesmo conjunto de features para treino e teste

### 3. **Modelagem Balanceada (MELHORADO)**
- **Random Forest** com class_weight='balanced' e hiperparâmetros otimizados
- **XGBoost** com configurações balanceadas e paralelização
- **Logistic Regression** com regularização e balanceamento
- **Validação Cruzada** com StratifiedKFold e métrica F1-score
- **Configurações anti-overfitting:** max_depth limitado, min_samples aumentado

### 4. **Otimização de Threshold**
- **Threshold Optimization** para maximizar F1-score
- **Análise de distribuição** de predições
- **Múltiplas submissões** com diferentes thresholds
- **Threshold ideal** baseado na distribuição do treino

### 5. **Geração de Submissão Inteligente**
- Uso correto do business_id do conjunto de teste
- Formato correto: business_id, destaque
- Análise de distribuição de predições
- Múltiplas versões para teste
- Estatísticas detalhadas das predições


## 🎓 Conclusão

Este notebook implementa uma solução para o desafio de previsão de locais altamente avaliados em Toronto. 

**Resumo do Desafio — Predict Highly Rated Venues**

Implementei pipeline limpo e otimizado com: EDA, feature engineering (análise de sentimento das reviews com método simplificado, TF-IDF, features temporais/sazonais, features geográficas), pré-processamento consistente treino/teste, modelagem balanceada (Random Forest, XGBoost e Logistic Regression com class_weight='balanced'), validação cruzada com StratifiedKFold e métrica F1-score (Kaggle), otimização automática de threshold. Gerei múltiplas submissões: submission_best_model.csv (melhor modelo com threshold otimizado) — score: 0.7358; submission_rf_optimized.csv (Random Forest otimizado) — score: 0.7358. Melhorias implementadas: análise de sentimento, features temporais, modelagem balanceada com XGBoost, otimização de threshold, código limpo e otimizado.


